# High Priority Bugs for PyTorch


Requires PyTorch Version: 1.10.0

In [9]:
!pip install torch==1.10.0

In [10]:
print(torch.__version__)

1.10.0+cu111


## Bug 1-3
`torch.{h, v, d}split` crash when sections = 0. https://github.com/pytorch/pytorch/issues/69270

In [ ]:
import torch
tensor = torch.rand(torch.Size([2, 2, 4]), dtype=torch.float32)
sections = 0
res1 = torch.dsplit(tensor, sections)

In [ ]:
import torch
tensor = torch.rand(torch.Size([2, 2, 4]), dtype=torch.float32)
sections = 0
res1 = torch.hsplit(tensor, sections)

In [ ]:
import torch
tensor = torch.rand(torch.Size([2, 2, 4]), dtype=torch.float32)
sections = 0
res1 = torch.vsplit(tensor, sections)

## Bug 4
`max_unpool2d` returns a tensor with negative dimension. https://github.com/pytorch/pytorch/issues/73154

In [ ]:
import torch
kernel_size = [3, 2, 2]
output_size = [0, 1, 1]
input = torch.rand([20, 16, 50, 44, 31], dtype=torch.float32)

torch.nn.FractionalMaxPool3d(kernel_size,output_size=output_size)(input)
# segmentation fault (core dumped)

## Bug 5-6
`AdaptiveAvgPool{2|3}d` should not create tensor with negative dimension. https://github.com/pytorch/pytorch/issues/70232

In [ ]:
import torch
output_size = [-36, 0]
arg_class = torch.nn.AdaptiveAvgPool2d(output_size)
tensor = torch.rand([128, 2048, 4, 4], dtype=torch.float32)
result = arg_class(tensor)
print(result.shape)
# torch.Size([128, 2048, -36, 0])

In [ ]:
import torch
output_size = [-36, 0, 0]
arg_class = torch.nn.AdaptiveAvgPool3d(output_size)
tensor = torch.rand([4, 4, 128, 2048, 4], dtype=torch.float32)
result = arg_class(tensor)
print(result.shape)
# torch.Size([4, 4, -36, 0, 0])

In [ ]:
import torch
size = [2, 3]
stride = [-1, 2]
res = torch.empty_strided(size,stride,)
print(torch.sum(res))
# tensor(nan)
print(res.shape)
# torch.Size([2, 3])
print(res)
# RuntimeError: setStorage: sizes [6], strides [2], storage offset 0, and itemsize 4 requiring a storage size of 44 are out of bounds for storage of size 16

## Bug 7-8
`torch.{addcmul,addcdiv}` triggers INTERNAL ASSERT FAILED. https://github.com/pytorch/pytorch/issues/70486

In [ ]:
import torch
arg_1 = torch.rand([5, 5], dtype=torch.float64)
arg_2 = torch.rand([5, 5], dtype=torch.float64)
arg_3 = torch.rand([1, 5], dtype=torch.complex128)
res = torch.addcmul(arg_1,arg_2,arg_3)
# RuntimeError: !(has_different_input_dtypes && !config.promote_inputs_to_common_dtype_ && (has_undefined_outputs || config.enforce_safe_casting_to_output_ || config.cast_common_dtype_to_outputs_))INTERNAL ASSERT FAILED at "../aten/src/ATen/TensorIterator.cpp":331, please report a bug to PyTorch. 

In [ ]:
import torch
arg_1 = torch.rand([1], dtype=torch.complex128)
arg_2 = torch.rand([5, 5, 1], dtype=torch.complex128)
arg_3 = torch.rand([1, 3], dtype=torch.float32)
res = torch.addcdiv(arg_1,arg_2,arg_3,)
# RuntimeError: !(has_different_input_dtypes && !config.promote_inputs_to_common_dtype_ && (has_undefined_outputs || config.enforce_safe_casting_to_output_ || config.cast_common_dtype_to_outputs_))INTERNAL ASSERT FAILED at "../aten/src/ATen/TensorIterator.cpp":331, please report a bug to PyTorch. 

## Bug 9
`torch.as_strided` can create a tensor with negative dimension. https://github.com/pytorch/pytorch/issues/70672

In [ ]:
import torch
a = torch.rand([3, 3])
b = torch.as_strided(a, [1, -1], [1, 1])
print(b.shape)
# torch.Size([1, -1])

## Bug 10
`Tensor.baddbmm_` unexpectedly succeeds. https://github.com/pytorch/pytorch/issues/71085

In [ ]:
import torch
results = dict()
input = torch.rand([10, 3, 5, 0], dtype=torch.float32)
batch1 = torch.rand([10, 3, 4], dtype=torch.float32)
batch2 = torch.rand([10, 4, 5], dtype=torch.float32)
input.clone().baddbmm(batch1, batch2)
# RuntimeError: expand(torch.FloatTensor{[10, 3, 5, 0]}, size=[10, 3, 5]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)
input.clone().baddbmm_(batch1, batch2)
# succeed

## Bug 11
`embedding_bag` will trigger segmentation fault in Linux. https://github.com/pytorch/pytorch/issues/71094

In [ ]:
import torch

input = torch.randint(-128, 128, [3, 6], dtype=torch.int64)
weight = torch.rand([30522, 384], dtype=torch.float32)
padding_idx = 0
max_norm = None
norm_type = 2.0
scale_grad_by_freq = False
sparse = False
torch.nn.functional.embedding_bag(
    input,
    weight,
    max_norm=max_norm,
    norm_type=norm_type,
    scale_grad_by_freq=scale_grad_by_freq,
    sparse=sparse,
    padding_idx=padding_idx,
)

## Bug 12
`torch.set_default_dtype` will crash with `complex` dtype. https://github.com/pytorch/pytorch/issues/71674

In [ ]:
import torch
d = torch.complex64
torch.set_default_dtype(d)
# segmentation fault

## Bug 13
`tril_indices` and `triu_indices` have strange behavior when `row=0`. https://github.com/pytorch/pytorch/issues/72900

In [3]:
import torch
results = dict()
row = 0
col = 1
offset = 2
try:
  results["res_1"] = torch.tril_indices(row, col, offset=offset)
except Exception as e:
  results["err_1"] = str(e)
try:
  results["res_2"] = torch.triu_indices(row,col,offset=offset)
except Exception as e:
  results["err_2"] = str(e)

print(results)

{'res_1': tensor([[0],
        [0]]), 'err_2': 'Trying to create tensor with negative dimension -1: [2, -1]'}


## Bug 14-15
`irfft2` and `irfftn` INTERNAL ASSERT FAILED. https://github.com/pytorch/pytorch/issues/72910

In [4]:
import torch
results = dict()
input = torch.rand([2, 1])
torch.fft.irfftn(input)
# RuntimeError: out_size == signal_size[i + 1] || out_size == (signal_size[i + 1] / 2) + 1INTERNAL ASSERT FAILED at "../aten/src/ATen/native/mkl/SpectralOps.cpp":458, please report a bug to PyTorch
torch.fft.irfft2(input)
# RuntimeError: out_size == signal_size[i + 1] || out_size == (signal_size[i + 1] / 2) + 1INTERNAL ASSERT FAILED at "../aten/src/ATen/native/mkl/SpectralOps.cpp":458, please report a bug to PyTorch.

RuntimeError: ignored

In [ ]:
import torch
input = torch.rand([1, 1, 2, 2], dtype=torch.float32)
indices = torch.randint(-16,1024,[1, 1, 2, 2], dtype=torch.int64)
kernel_size = [16, -1024]
stride = [-16, 1]
print(torch.nn.functional.max_unpool2d(input, indices, kernel_size, stride))
# tensor([], size=(1, 1, 0, -1023))

## Bug 16
`torch.kthvalue` returns random value when the k is invalid. https://github.com/pytorch/pytorch/issues/68813

In [ ]:
import torch
input = torch.tensor([0,1,2,3,4])
k = 6
res1 = torch.kthvalue(input, k)
res2 = input.kthvalue(k)
print(res1)
print(res2)

## Bug 17
`CrossEntropyLoss` triggers floating point exception. https://github.com/pytorch/pytorch/issues/73165

In [ ]:
import torch
results = dict()
input_1 = torch.rand([5, 0], dtype=torch.float32)
input_2 = torch.rand([5, 0], dtype=torch.float32)

torch.nn.CrossEntropyLoss()(input_1, input_2)
# floating point exception

## Bug 18
`LayerNorm` triggers INTERNAL ASSERT. https://github.com/pytorch/pytorch/issues/73174

In [ ]:
import torch
normalized_shape = [1024]
layer = torch.nn.LayerNorm(normalized_shape)
input = torch.randint(0,8, [0, 1, 1024])
res = layer(input)
# RuntimeError: isDifferentiableType(variable.scalar_type())INTERNAL ASSERT FAILED at "../torch/csrc/autograd/functions/utils.h":65, please report a bug to PyTorch.

## Bug 19-20
`FractionalMaxPool{2,3}d` trigger segmentation fault when `output_size` contains 0. https://github.com/pytorch/pytorch/issues/73624

In [ ]:
import torch
kernel_size = [2, 2]
output_size = [0, 1]
input = torch.rand([16, 50, 44, 31], dtype=torch.float32)

torch.nn.FractionalMaxPool2d(kernel_size,output_size=output_size)(input)
# segmentation fault (core dumped)

## Bug 21
Many APIs `INTERNAL ASSERT FAILED` when promoting `complex32` dtype. https://github.com/pytorch/pytorch/issues/71671. https://github.com/pytorch/pytorch/issues/71680.

In [5]:
import torch
results = dict()
input = torch.rand([1], dtype=torch.float64)
other = torch.rand([1], dtype=torch.complex32)
try:
  results["res"] = torch.add(input, other)
except Exception as e:
  results["err"] = "ERROR:"+str(e)
print(results)

{'err': 'ERROR:common_dtype_ != ScalarType::UndefinedINTERNAL ASSERT FAILED at "../aten/src/ATen/TensorIterator.cpp":236, please report a bug to PyTorch. '}


## Bug 22
`torch.broadcast_to` can create tensor with negative dimension. https://github.com/pytorch/pytorch/issues/70398

In [6]:
import torch
input = torch.rand([3])
shape = [-2, 3]
res = torch.broadcast_to(input,shape,)
print(res.shape)
# torch.Size([-2, 3])
print(torch.sum(res))
# tensor(0.)
torch.all(res)
# RuntimeError: Trying to create tensor with negative dimension -2: [-2, 3]

torch.Size([-2, 3])
tensor(0.)


RuntimeError: ignored

In [7]:
import torch
input = torch.rand([3])
shape = [-2, 3]
res = input.expand(shape)
print(res.shape)
print(torch.sum(res))
torch.all(res)

torch.Size([-2, 3])
tensor(0.)


RuntimeError: ignored

## Bug 23
`torch.empty_strided` works when the stride is negative. https://github.com/pytorch/pytorch/issues/70397

In [8]:
import torch
size = [2, 3]
stride = [-1, 2]
res = torch.empty_strided(size,stride,)
print(torch.sum(res))
# tensor(nan)
print(res.shape)
# torch.Size([2, 3])
print(res)
# RuntimeError: setStorage: sizes [6], strides [2], storage offset 0, and itemsize 4 requiring a storage size of 44 are out of bounds for storage of size 16

tensor(nan)
torch.Size([2, 3])


RuntimeError: ignored